<a href="https://colab.research.google.com/github/Oleksandr190378/data-computing/blob/main/conversation_history.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -qU langchain langchain_community langchain_openai python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.9/399.9 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.1/292.1 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.1/376.1 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141

In [12]:
from os import environ
from langchain.chains.conversation.memory import ConversationSummaryBufferMemory
from langchain import LLMChain, PromptTemplate
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

In [3]:
import getpass

OPENAI_API_KEY = getpass.getpass(prompt='Введіть ваш OpenAI API ключ: ')

print("API ключ успішно введений!")

Введіть ваш OpenAI API ключ: ··········
API ключ успішно введений!


In [4]:
import os
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [5]:
template = """
You are a helpful assistant. Help the user with their requests.
"""

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", template),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{query}"),
    ]
)


In [13]:
memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=500, memory_key="history", return_messages=True)

<ipython-input-13-7e5cb334991c>:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=500, memory_key="history", return_messages=True)


In [14]:
llm = ChatOpenAI(model="gpt-4o-mini")

In [15]:
store = {}

def get_session_history(session_id: str):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

session_id = "bcd"

In [16]:
chain = prompt_template | llm
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="query",
    history_messages_key="history"
)

In [17]:
config = {"configurable": {"session_id": session_id}}

In [18]:
response = chain_with_history.invoke({"query": "Hi GPT, i am going to market in the evening and i need to buy milk, bread, almonds and bananas. Can you remember it for me?"}, config=config)
print(response)

content="I can't remember things for you, but I can help you create a list! Here's what you need to buy:\n\n- Milk\n- Bread\n- Almonds\n- Bananas\n\nYou can write this down or save it on your phone. Let me know if you need anything else!" additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 58, 'prompt_tokens': 56, 'total_tokens': 114, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_1bb46167f9', 'finish_reason': 'stop', 'logprobs': None} id='run-8636f933-ec72-4472-a639-f77cce70e86c-0' usage_metadata={'input_tokens': 56, 'output_tokens': 58, 'total_tokens': 114}


In [19]:
response = chain_with_history.invoke({"query": "Can you remind me all items i need to buy today?"}, config=config)
print(response)

content="I can't send reminders, but I can help you list out the items you need to buy. Here they are:\n\n- Milk\n- Bread\n- Almonds\n- Bananas\n\nYou might want to set a reminder on your phone or write it down so you don't forget. Let me know if you need help with anything else!" additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 67, 'prompt_tokens': 134, 'total_tokens': 201, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_3a215618e8', 'finish_reason': 'stop', 'logprobs': None} id='run-e2dfdbec-a622-4fa3-9879-8e3b01ff2d8d-0' usage_metadata={'input_tokens': 134, 'output_tokens': 67, 'total_tokens': 201}


In [20]:
history = get_session_history(session_id).messages
print("History for session_id:", session_id)
for message in history:
    print(message)

History for session_id: bcd
content='Hi GPT, i am going to market in the evening and i need to buy milk, bread, almonds and bananas. Can you remember it for me?' additional_kwargs={} response_metadata={}
content="I can't remember things for you, but I can help you create a list! Here's what you need to buy:\n\n- Milk\n- Bread\n- Almonds\n- Bananas\n\nYou can write this down or save it on your phone. Let me know if you need anything else!" additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 58, 'prompt_tokens': 56, 'total_tokens': 114, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_1bb46167f9', 'finish_reason': 'stop', 'logprobs': None} id='run-8636f933-ec72-4472-a639-f77cce70e86c-0' usage_metadata={'input_tokens': 56, 'output_tokens': 58, 'total_tokens': 114}
content='Can you remind me all items i need to buy today?' additional_kwargs={} response_metadata={}
content="I can'

In [21]:
response = chain_with_history.invoke({"query": "write me a healthy meal plan for tomorrow"}, config=config)
response.content

"Here's a healthy meal plan for tomorrow:\n\n### Breakfast\n- **Oatmeal** topped with fresh berries (strawberries, blueberries) and a tablespoon of almond butter.\n- A glass of water or herbal tea.\n\n### Snack\n- A small handful of **almonds** (about 1 ounce).\n- An apple or a banana.\n\n### Lunch\n- **Grilled chicken salad** with mixed greens, cherry tomatoes, cucumber, avocado, and a light vinaigrette dressing.\n- A slice of whole-grain bread.\n\n### Snack\n- Carrot and cucumber sticks with **hummus**.\n\n### Dinner\n- **Baked salmon** with lemon and herbs.\n- Steamed broccoli or asparagus on the side.\n- A small serving of quinoa or brown rice.\n\n### Dessert (optional)\n- A small bowl of Greek yogurt with honey and a sprinkle of cinnamon.\n\nFeel free to adjust portion sizes and ingredients based on your preferences and dietary needs! Enjoy your meals!"

In [35]:
def chat_with_history(query):
    try:
        response = chain_with_history.invoke({"query": query}, config=config)
        content = response.content if hasattr(response, 'content') else str(response)
        history = get_session_history(session_id).messages
        history_text = "\n".join([
          f"{msg.type}: {msg.content}" if hasattr(msg, 'type') else f"{msg.role}: {msg.content}"
          for msg in history
        ])
        return content, history_text
    except Exception as e:
        return f"Error processing query: {e}", ""

In [36]:
query = "write me a healthy meal plan for tomorrow"
chat_with_history(query)

('Here’s another healthy meal plan for tomorrow:\n\n### Breakfast\n- **Overnight Oats**: Combine 1/2 cup rolled oats, 1 cup almond milk (or yogurt), 1 tablespoon chia seeds, and a handful of berries. Let it sit in the fridge overnight. In the morning, add a drizzle of honey or maple syrup if desired.\n- A cup of herbal tea or black coffee.\n\n### Snack\n- A small handful of **mixed nuts** or a piece of fruit, like a pear or a banana.\n\n### Lunch\n- **Mediterranean Chickpea Salad**: Combine canned chickpeas (rinsed), diced cucumbers, cherry tomatoes, red onion, parsley, and feta cheese with olive oil and lemon juice dressing.\n- A side of whole grain pita bread.\n\n### Snack\n- **Greek Yogurt** with a sprinkle of granola and a few slices of fresh fruit.\n\n### Dinner\n- **Grilled Chicken or Fish** (like salmon) seasoned with herbs and spices.\n- A side of roasted vegetables (such as zucchini, bell peppers, and carrots) and quinoa.\n\n### Dessert (optional)\n- A small bowl of **fresh be

In [23]:
!pip install -qU  gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 9.0 MB/s eta 0:00:00


In [37]:
import gradio as gr
iface = gr.Interface(
    fn=chat_with_history,
    inputs="text",
    outputs=["text", "text"],
    title="Chat with History",
    description="Interact with the assistant and view the chat history.",

)

In [38]:
iface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://7142ad650ea1920460.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
